In [1]:
import ee
import geemap
import geopandas as gpd
import numpy as np
from agroforestry.config import * 
from agroforestry.geeHelpers import *
from agroforestry.naipProcessing import *
from agroforestry.snicProcessing import *
from agroforestry.randomForest import *
from agroforestry.exportFunctions import *

In [2]:
#print
vsurfNoCor

['entropy_n',
 'contrast_n',
 'contrast_n_mean',
 'savg_g',
 'nd_mean_neighborhood',
 'contrast_g_mean',
 'nd',
 'contrast_g',
 'savg_n',
 'N']

In [3]:
# establish connection with ee account. might require some additional configuration based on local machine 
try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()# i

In [4]:
# type(grid24)
grid.Unique_ID
# grid24.Unique_ID
# gridSelect


0        X12-1
1        X12-2
2        X12-3
3        X12-4
4        X12-5
        ...   
768    X12-769
769    X12-770
770    X12-771
771    X12-772
772    X12-773
Name: Unique_ID, Length: 773, dtype: object

In [5]:
# define the aoi
aoiID = initGridID # something to itorate over for now is defined based on the input training dataset 
# or manually define it for where you want to apply the model too
# aoiID = "X12-601"
# # this becomes the AOI to used in the prepNAIP function. I'll need to edit it so that it converts the input data into a bbox 
# gridSelect = grid.loc[grid.Unique_ID == aoiID]


#Define bands to use
bandsToUse = vsurfNoCor

# select multiple grids level 1 
## want to pull thing from the csv rather than write it out. 
# aoiID = grid24
gridSelect =  grid.loc[grid.Unique_ID.isin(grid36.Unique_ID)].dissolve()

# after the desolve this gets assign a unique id some way. Might want to assign it the initGridID instead 
# might want to 
len(gridSelect.Unique_ID)

# len(grid8)



1

In [6]:
grid24

,Unnamed: 0,Unique_ID,poisition
0,1,X12-111,1
1,2,X12-112,1
2,3,X12-113,1
3,4,X12-149,1
4,5,X12-150,1
5,6,X12-151,1
6,7,X12-187,1
7,8,X12-188,1
8,9,X12-189,1
9,10,X12-71,2


In [7]:
# convert to a gee object 
aoi1 = geemap.gdf_to_ee(gridSelect)
# create a sub grid for downloading 
# downloadGrids = geemap.fishnet(aoi1.geometry(), rows=6, cols=4, delta=0)
aoiID

'X12-150'

In [8]:
Map = geemap.Map(center=(42.3, -98), zoom=10)
Map.add_basemap('HYBRID')
Map.addLayer(aoi1, {'color': '000000ff',
                    'width': 2,
                    'lineType': 'solid'},
             'area of interest')
# Map.addLayer(downloadGrids, {'color': '000000ff',
#                     'width': 2,
#                     'lineType': 'solid'},
#              'area of subgrid')

Map

Map(center=[42.3, -98], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(c…

In [9]:
# import training dataset 
# trainingData = gpd.read_file(filename="data/processed/trainingdataset_withClasses.geojson")
trainingData = gpd.read_file(filename="data/processed/" + str(initGridID) +"/"+ "agroforestrySamplingData_"+str(year)+".geojson") # initGridID defined int he config file

# print(type(trainingData))
# select the training class of interest and drop unnecessary columns
trainingSubset = trainingData
# trainingSubset =  trainingData[trainingData.sampleStrat == "original"] ## will want to drop this as we wont have multiple sampling categories 
# print(trainingSubset)
# convert to ee object
pointsEE = geemap.gdf_to_ee(gdf=trainingSubset)
# subset testing and training data 
training = pointsEE.filter(ee.Filter.gt('random', test_train_ratio))
testing = pointsEE.filter(ee.Filter.lte('random',test_train_ratio))
# traing the rf model 
# rfCluster = trainRFModel(bands=bandsToUse_Cluster, inputFeature=training, nTrees=nTrees,setSeed=setSeed)
# rfPixel = trainRFModel(bands=bandsToUse_Pixel, inputFeature=training, nTrees=nTrees,setSeed=setSeed)
rfPixelTrim = trainRFModel(bands=bandsToUse,  inputFeature=training, nTrees=nTrees,setSeed=setSeed )
## run validation using the testing set 
# clusterValidation = testRFClassifier(classifier=rfCluster, testingData= testing)
# pixelValidation = testRFClassifier(classifier=rfPixel, testingData= testing)
pixelValidationTrim = testRFClassifier(classifier=rfPixelTrim, testingData= testing)


In [10]:
trainingData



,B,B_mean,G,G_mean,N,N_mean,R,R_mean,contrast_g,contrast_g_mean,...,nd_mean,nd_mean_neighborhood,nd_sd_neighborhood,presence,savg_g,savg_g_mean,savg_n,savg_n_mean,random,geometry
0,38,43.166061,61,72.667152,166,139.971725,33,45.089195,209.791762,245.847931,...,0.516225,0.632426,0.059610,1.0,124.214212,152.158188,303.612362,281.219147,0.196103,POINT (-96.51443 40.59645)
1,46,44.237465,74,73.076698,152,157.038345,40,41.679939,213.990177,212.368927,...,0.586384,0.615984,0.057387,1.0,119.203412,139.130539,275.557790,280.684448,0.366014,POINT (-96.51497 40.59644)
2,43,34.765129,68,43.404697,128,69.450768,43,27.477869,207.056756,148.781113,...,0.377278,0.299952,0.146495,1.0,91.854665,94.339981,125.100069,155.428665,0.340727,POINT (-96.51513 40.59620)
3,35,37.736633,56,52.164356,103,95.778221,29,32.784157,243.676585,226.250641,...,0.457756,0.386283,0.186196,1.0,108.487822,113.220764,183.379596,206.416382,0.735450,POINT (-96.51510 40.59593)
4,38,40.421646,65,65.317398,108,126.671982,32,38.826694,229.202378,306.224884,...,0.521424,0.489454,0.119309,1.0,116.954906,134.889648,212.648093,244.836319,0.889373,POINT (-96.51350 40.59637)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700,48,46.841942,84,85.527878,139,145.591812,60,58.598465,25.970875,55.319492,...,0.428424,0.393930,0.055670,0.0,172.216165,172.690872,280.935662,290.959473,0.411756,POINT (-96.40741 40.53161)
701,40,40.853512,72,71.142998,178,180.416550,34,36.201294,11.481675,19.264711,...,0.665871,0.679557,0.028812,0.0,140.663890,145.260483,362.391889,357.264343,0.478906,POINT (-96.40775 40.53446)
702,41,41.773705,77,73.076920,180,180.335419,39,38.002682,20.741670,24.004387,...,0.652443,0.662607,0.047802,0.0,144.139074,148.104019,362.291073,359.303314,0.003170,POINT (-96.40741 40.53451)
703,49,43.136452,76,75.169121,174,179.951309,44,40.607304,29.992475,35.205219,...,0.632541,0.644389,0.044731,0.0,147.787282,156.262543,355.657341,354.094482,0.250454,POINT (-96.40826 40.53446)


In [11]:
# cant print tuple with this function
geePrint(pixelValidationTrim)

0.9629629629629629


In [12]:

# define export aoi
# exportAOI = ee.Feature(downloadGrids.toList(50).get(4))
# geePrint(exportAOI)
# exportAOI.geometry()
year


2020

In [13]:
### 
# Generate model based on year define in config 

# generate NAIP layer 
naipEE = prepNAIP(aoi=aoi1, year=year,windowSize=windowSize)
geePrint(naipEE.bandNames())
# normal the naip data
# normalizedNAIP = normalize_by_maxes(img=naipEE, bandMaxes=bandMaxes)

# produce the SNIC object 
## filtering the image bands right away based on the single model output 
snicData = snicOutputs(naip = naipEE,
                       SNIC_SeedShape = SNIC_SeedShape, 
                       SNIC_SuperPixelSize = SNIC_SuperPixelSize, 
                       SNIC_Compactness = SNIC_Compactness, 
                       SNIC_Connectivity = SNIC_Connectivity,
                       # nativeScaleOfImage = nativeScaleOfImage, 
                       bandsToUse_Cluster = bandsToUse_Cluster).select(bandsToUse)
# apply the model and clip to aoi and reclass to unsigned 8bit image 
classifiedPixelsTrim = applyRFModel(imagery=snicData, bands=bandsToUse,classifier=rfPixelTrim).clip(aoi1).uint8()
# produce image for map 
demoImage = classifiedPixelsTrim #.clip(exportAOI)
# demoImage = classifiedPixelsTrim.clip(ee.Feature(downloadGrids.toList(50).get(10))).reproject(crs='EPSG:4326', scale=5)

geePrint(demoImage)


['R', 'G', 'B', 'N', 'savg_g', 'contrast_g', 'entropy_g', 'savg_n', 'contrast_n', 'entropy_n', 'nd', 'nd_sd_neighborhood', 'nd_mean_neighborhood']
{'type': 'Image', 'bands': [{'id': 'classification', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


In [14]:
# print
# vsurfNoCor
# vsurfWithCor

In [15]:
# Set visualization parameters.
vis_params = {
    'min': 0,
    'max': 1,
    'palette': ['#f5f7f710', '#10c9a1'],
}
# add features to the map
Map.addLayer(demoImage, vis_params, str(year) + ' model')
# Map.addLayer(demoImage2010, vis_params,'2010 model',False)
# Map.addLayer(demoImage2020, vis_params,'2020 model',False)

Map

Map(center=[42.3, -98], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(c…

In [17]:
# export image to asset 
task = ee.batch.Export.image.toAsset(
  image = demoImage,
  description = str(initGridID) + "_" + str(year) + "_36grid",
  assetId = "projects/agroforestry2023/assets/"+ str(initGridID) + "_" + str(year) + "_36grid",
  region=aoi1.geometry(),
  scale=1,
  crs= demoImage.projection(),
  maxPixels = 1e13
)
task.start()

In [16]:
str(initGridID)
str(year)

'2020'

In [ ]:
# track the task 
import time
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)

In [ ]:
geePrint(demoImage)